<a href="https://colab.research.google.com/github/julx134/Amazon-Recommendation-System/blob/main/Shilling_Detector_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>